In [1]:
from gensim.models import FastText
import gensim
import pandas as pd

DATA_PATH = '/data/'
MODEL_PATH = DATA_PATH + 'models/ft1_ru/'
SOURCE_PATH = DATA_PATH + 'captions/clean/'

cities = ['moscow', 'spb']#, 'nyc', 'london']
years = ['2016', '2017', '2018', '2019', '2020']
files = []
for city in cities:
    for year in years:
        files.append([city, year])

def csv_path(path, city, year):
    return path + city + '_posts_' + year + '.csv'

valid_langs = set(['__label__ru'])

USE_PRETRAINED_MODEL = True

In [3]:
!mkdir {MODEL_PATH}
class MyCorpus(object):
    iter = 0
    def __iter__(self):
        self.iter += 1
        for city in cities:
            for year in years:
                df = pd.read_csv(csv_path(SOURCE_PATH, city, year))
                df = df[df.lang.isin(valid_langs)]
                for s in df['caption']:
                    yield s.split()
                              
            print(f'iter {self.iter} for {city} completed')
                
        print(f'\niter {self.iter} completed\n')

sentences = MyCorpus()

# dimension 300, with character n-grams of length 5, a window of size 5 and 10 negatives
# https://fasttext.cc/docs/en/crawl-vectors.html

if USE_PRETRAINED_MODEL:
    model =  gensim.models.fasttext.load_facebook_model(MODEL_PATH + 'wiki.ru.bin')
else:
    model = FastText(size=300, window=5, min_count=5, word_ngrams=5, negative=10, workers=35)
print(f'success, vocab: {len(model.wv.vocab)}')

mkdir: cannot create directory ‘/data/models/ft1_ru/’: File exists
success, vocab: 1888423


In [4]:
print('building vocab\n')
model.build_vocab(sentences=MyCorpus(), update=True)
total_examples = model.corpus_count
print(f'\ncorpus count: {total_examples}; vocab: {len(model.wv.vocab)}\nend of building')

building vocab

iter 1 for moscow completed
iter 1 for spb completed

iter 1 completed


corpus count: 32652789; vocab: 4192061
end of building


In [4]:
print(len(model.wv.vocab))

1888423


In [ ]:
print('start training\n')
model.train(sentences=MyCorpus(), total_examples=total_examples, epochs=10)
print(f'\nend of training')

start training



In [ ]:
model.save(MODEL_PATH + 'mdl')

f = open(MODEL_PATH + 'description.txt', 'w')
f.write('date: 08.05\n')
f.write('params: size=300, min_count=1, window=5, negative=10, word_ngrams=5, epochs=5\n')
f.write(f'sourse: {SOURCE_PATH}\n')
f.write('cities: moscow, spb; only russian\n')
print('success')
f.close()

In [9]:
with open('tests.txt') as f:
    tests = [[word for word in line.split()] for line in f]

for test in tests:
    print(model.wv.similarity(test[0], test[1]), test[0], test[1])
    

0.53660005 8марта 23февраля
0.22803883 8марта выставка
0.27682415 8марта рождество
0.3431545 8марта свадьба
0.22072798 8марта 9мая
0.124956414 8марта сплин
0.33541226 сплин слот
0.4532915 сплин rammstein
0.24941845 сплин рождество
0.1894283 сплин свадьба
0.28343615 сплин маникюр
0.21619888 сплин спб
0.5292578 сплин ддт
0.48762012 сплин концерт
0.3917424 сплин песня
0.40032834 сплин нойз
0.42289874 зенит ска
0.286868 зенит хоккей
0.27438378 зенит баскетбол
0.22971313 зенит сплин
0.13205908 зенит 8марта
0.13331057 зенит рождество
0.28403854 зенит футбол
0.2957577 зенит арена
0.27713022 зенит мяч
0.22643316 зенит гол
0.5436301 зенит спартак
0.19315839 зенит маникюр
0.14836843 зенит сушь
0.19022311 зенит тенис
0.26732105 тенис волебол
0.20677681 тенис марафон
0.22909734 тенис лыжа
0.23921612 зенит спорт
0.19260393 тенис спорт
0.56570345 баскетбол спорт
0.439621 итмо университет
False


In [14]:
print('9мая' in model.wv.vocab)

False
